<a href="https://colab.research.google.com/github/nalivaikaanastasiya-dev/Hackathon_2/blob/main/Hackaton_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Fine-Tuned LLM for Sentiment Analysis and Contextual Responses**

# **Step 1: Define the Goal & Dataset**

**Goal:**


*   Classify the sentiment of user text (positive / negative).
*   Generate a context-aware response based on both user input and retrieved external information.



**Domain Choice:**
Movie reviews

**Dataset:**


*   IMDB Movie Reviews Dataset
*   Binary sentiment labels: positive, negative
*   Subsample: 10,000 examples to keep training lightweight.




In [ ]:
from datasets import load_dataset

dataset = load_dataset("imdb")

dataset["train"] = dataset["train"].shuffle(seed=42).select(range(10000))
dataset["test"] = dataset["test"].shuffle(seed=42).select(range(2000))

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

# **Step 2: Choose a Base Model & Load Data**

**Base Model:**
distilbert-base-uncased


*   Lightweight
*   Fast on CPU
*   Good performance for classification



**Tokenization**

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=256)

tokenized_dataset = dataset.map(tokenize, batched=True)
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
tokenized_dataset.set_format("torch")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

# **Step 3: Parameter-Efficient Fine-Tuning with LoRA**

**Why LoRA?**


*   Only trains small low-rank matrices
*   Freezes base model weights
*   Memory-efficient and fast


**LoRA Configuration**

In [ ]:
from peft import LoraConfig, get_peft_model
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2
)

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_lin", "v_lin"],
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_CLS"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 739,586 || all params: 67,694,596 || trainable%: 1.0925


# **Step 4: Train & Validate the Classifier**

**Training Setup**

In [ ]:
import torch
import numpy as np
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)

    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average="binary"
    )
    acc = accuracy_score(labels, preds)

    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    fp16=torch.cuda.is_available(),
    logging_dir="./logs",
    save_strategy="epoch",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics
)

trainer.train()

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.328800,0.296342,0.881500,0.879602,0.884000,0.881796


TrainOutput(global_step=1250, training_loss=0.35686854858398437, metrics={'train_runtime': 10226.5735, 'train_samples_per_second': 0.978, 'train_steps_per_second': 0.122, 'total_flos': 673697034240000.0, 'train_loss': 0.35686854858398437, 'epoch': 1.0})

**Metrics:**

*   Accuracy
*   Validation loss



# **Step 5: Build a Retrieval Component**

**Embeddings Model**

sentence-transformers/all-MiniLM-L6-v2

In [ ]:
!pip install faiss-cpu sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 60.1 MB/s eta 0:00:00


In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

embedder = SentenceTransformer("all-MiniLM-L6-v2")

corpus = dataset["train"]["text"][:2000]

embeddings = embedder.encode(
    corpus,
    batch_size=16,
    show_progress_bar=True,
    convert_to_numpy=True
)

index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

print("FAISS index size:", index.ntotal)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/125 [00:00<?, ?it/s]

FAISS index size: 2000


**Retrieval Function**

In [ ]:
def retrieve_context(query, k=3):
    q_emb = embedder.encode([query])
    distances, indices = index.search(q_emb, k)
    return [corpus[i] for i in indices[0]]

# **Step 6: Generate Context-Aware Responses**

**Prompt Construction**

In [ ]:
def build_prompt(user_input, context, sentiment):
    context_text = "\n".join(context)
    return f"""
User input: {user_input}
Detected sentiment: {sentiment}

Relevant context:
{context_text}

Generate a helpful and empathetic response.
"""

**Response Generation**

In [ ]:
user_input = "The movie was boring and too long"
sentiment = "negative"
context = ["Example context sentence 1", "Example context sentence 2"]

prompt = build_prompt(user_input, context, sentiment)

In [ ]:
user_input = "The movie was boring and too long"

sentiment = "negative"

context = [
    "The reviewer felt the movie was too long.",
    "The plot failed to keep the audience engaged."
]

prompt = build_prompt(user_input, context, sentiment)

from transformers import AutoModelForCausalLM, AutoTokenizer

gen_tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
gen_model = AutoModelForCausalLM.from_pretrained("distilgpt2")

inputs = gen_tokenizer(prompt, return_tensors="pt")
outputs = gen_model.generate(
    **inputs,
    max_length=200,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    repetition_penalty=1.2,
    no_repeat_ngram_size=3,
    pad_token_id=gen_tokenizer.eos_token_id
)

response = gen_tokenizer.decode(outputs[0], skip_special_tokens=True)

print(response)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]


User input: The movie was boring and too long
Detected sentiment: negative

Relevant context:
The reviewer felt the movie was too long.
The plot failed to keep the audience engaged.

Generate a helpful and empathetic response.
See also:


# **Step 7:  Sentiment-Aware Assistant: Streamlit + ngrok**

In [ ]:
!pip install -q streamlit pyngrok transformers torch

app_code = r"""
import streamlit as st
from transformers import AutoTokenizer, AutoModelForCausalLM
import time
import os
import csv

# --- Простое retrieval ---
CORPUS = [
    "The reviewer felt the movie was too long.",
    "The plot failed to keep the audience engaged.",
    "The characters felt flat and underdeveloped.",
    "The cinematography was visually impressive."
]

def retrieve_context(text, k=2):
    text = text.lower()
    hits = [c for c in CORPUS if any(w in c.lower() for w in text.split())]
    return hits[:k] if hits else CORPUS[:k]

# --- Prompt builder (строгий, одна строка) ---
def build_prompt(user_text, context):
    return (
        f"Answer in 2–3 sentences using ONLY the information explicitly stated in the Context. "
        f"Do NOT add new facts, opinions, examples, or references. "
        f"If something is not mentioned in the Context, do not mention it. "
        f"Context: {' '.join(context)} Review: {user_text}"
    )

tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
model = AutoModelForCausalLM.from_pretrained("distilgpt2")

def generate_response(prompt):
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(
        **inputs,
        max_new_tokens=80,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.5,
        no_repeat_ngram_size=3,
        pad_token_id=tokenizer.eos_token_id
    )
    full_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Убираем prompt из вывода
    response = full_text[len(prompt):].strip()
    return response

# --- Streamlit UI ---
st.title("Sentiment-Aware Assistant")

user_text = st.text_area("Enter your text:")

if st.button("Analyze"):
    sentiment = "Negative" if any(w in user_text.lower() for w in ["boring", "bad", "long"]) else "Positive"
    context = retrieve_context(user_text)
    prompt = build_prompt(user_text, context)

    start = time.time()
    response = generate_response(prompt)
    latency = time.time() - start

    st.write("**Sentiment:**", sentiment)
    st.markdown("### 💬 Response")
    st.success(response)
    st.write(f"⏱ Response time: {latency:.2f} seconds")

    # --- Feedback ---
    rating = st.slider("Rate the response", 1, 5)
    if st.button("Submit Feedback"):
        file_exists = os.path.isfile("feedback.csv")
        with open("feedback.csv", "a", newline="", encoding="utf-8") as f:
            writer = csv.writer(f)
            if not file_exists:
                writer.writerow(["user_text", "response", "rating"])
            writer.writerow([user_text, response, rating])
        st.success("Feedback submitted!")
"""

with open("streamlit_app.py", "w") as f:
    f.write(app_code)

import os, time
os.system("nohup streamlit run streamlit_app.py --server.port 8501 --server.address 0.0.0.0 &")
time.sleep(5)

from pyngrok import ngrok
ngrok.set_auth_token("37mtMomlOJqFcTz3iY78fVrGh5m_5okvZPABVgnb3m2pDYDZW")
public_url = ngrok.connect(8501)
print("Streamlit URL:", public_url)


Streamlit URL: NgrokTunnel: "https://melanospermous-quaternate-peg.ngrok-free.dev" -> "http://localhost:8501"
